We will need the game odds. Due to formatting it will be easiest to add it to team_data first. Below we import the odds dataset and join the betting line and odds. Note that the odds dataset uses abbreviations so we'll need to define those and add them to the data set in order to join

In [119]:
team_abbrv = {'Atlanta Braves' : 'ATL', 
              'Arizona Diamondbacks' : 'ARI', 
              'Baltimore Orioles' : 'BAL', 
              'Boston Red Sox' : 'BOS', 
              'Chicago Cubs' : 'CUB', 
              'Chicago White Sox' : 'CWS', 
              'Cincinnati Reds' : 'CIN', 
              'Cleveland Indians' : 'CLE', 
              'Colorado Rockies' : 'COL', 
              'Detroit Tigers' : 'DET',
              'Kansas City Royals': 'KAN', 
              'Houston Astros' : 'HOU', 
              'Los Angeles Angels' : 'LAA', 
              'Los Angeles Dodgers' : 'LAD', 
              'Miami Marlins' : 'MIA', 
              'Florida Marlins' : 'FLA', 
              'Milwaukee Brewers' : 'MIL', 
              'Minnesota Twins' : 'MIN', 
              'New York Mets' : 'NYM', 
              'New York Yankees' : 'NYY', 
              'Oakland Athletics' : 'OAK',
              'Philadelphia Phillies' : 'PHI', 
              'Pittsburgh Pirates' : 'PIT', 
              'San Diego Padres' : 'SDG', 
              'Seattle Mariners' : 'SEA', 
              'San Francisco Giants' : 'SFO', 
              'St. Louis Cardinals' : 'STL', 
              'Tampa Bay Rays' : 'TAM', 
              'Texas Rangers' : 'TEX', 
              'Toronto Blue Jays' : 'TOR', 
              'Washington Nationals' : 'WAS'}
team_data['TeamAbr'] = team_data['Team'].apply(lambda x: team_abbrv[x])

game_date = game_level[['GameID', 'DateTime']].astype({'GameID' : 'int64'})
game_date['Date'] = game_date['DateTime'].map(lambda x: x.month*100 + x.day)
game_date.drop(columns='DateTime', inplace=True)
team_data = team_data.merge(game_date, on='GameID')


odds = pd.read_csv(os.path.join(parentDirectory, 'data', 'mlbodds2019.csv'))
odds.rename(columns={'Team' : 'TeamAbr', 'Final' : 'Runs', 'Unnamed: 18' : 'Line Odds'}, inplace=True)
team_data = team_data.merge(odds[['Date', 'TeamAbr', 'Runs', 'Run Line', 'Line Odds', 'Pitcher']], how='left', on=['TeamAbr', 'Date', 'Runs'])

Now join the team data back to the game_level data in order to come up with our final dataset.

In [141]:
model_data = game_level[['GameID', 'AwayTeam', 'HomeTeam', 'DateTime', 'AwayScore', 'HomeScore']].astype({
    'AwayScore' : 'int',
    'HomeScore' : 'int'
})
model_data = model_data.merge(team_data, left_on=['GameID', 'AwayTeam'], right_on=['GameID', 'Team'])
model_data = model_data.merge(team_data, left_on=['GameID', 'HomeTeam'], right_on=['GameID', 'Team'], suffixes=('_away', '_home'))
model_data['ScoreDiff'] = model_data['AwayScore'] - model_data['HomeScore']
model_data.drop(columns=['Team_away', 'Team_home', 'Runs_away', 'Runs_home'], inplace=True)

We'd like to repeat these steps with the data from 2017 and 2018 as well. We've written them all into a function however so we'll let it do the heavy-lifting!

In [152]:
parsed18 = pkl.load(open(os.path.join(parentDirectory, 'data', 'parsedData2018.p'), 'rb'))
game_level18 = parsed18['Game']
team_level18 = parsed18['Team']
parsed17 = pkl.load(open(os.path.join(parentDirectory, 'data', 'parsedData2018.p'), 'rb'))
game_level17 = parsed17['Game']
team_level17 = parsed17['Team']
odds18 = pd.read_csv(os.path.join(parentDirectory, 'data', 'mlbodds2018.csv'))
odds17 = pd.read_csv(os.path.join(parentDirectory, 'data', 'mlbodds2017.csv'))
model_data18 = clean_data.clean_season_data(team_level18, game_level18, odds18)
model_data17 = clean_data.clean_season_data(team_level17, game_level17, odds17)
model_data = pd.concat([model_data, model_data18, model_data17])

In [155]:
model_data18.head(10)

,GameID,AwayTeam,HomeTeam,DateTime,AwayScore,HomeScore,GameNum_away,HomeAway_away,Hits_away,Errors_away,AB_away,RBI_away,BB_away,SO_away,PA_away,OBP_away,SLG_away,Starter_away,OBP_NUM_away,SLG_NUM_away,Runs_Mean_away,Hits_Mean_away,Errors_Mean_away,RBI_Mean_away,BB_Mean_away,SO_Mean_away,AB_Total_away,PA_Total_away,OBP_NUM_Total_away,SLG_NUM_Total_away,SLG_Mean_away,OBP_Mean_away,TeamAbr_away,Date_away,Run Line_away,Line Odds_away,Pitcher_away,GameNum_home,HomeAway_home,Hits_home,Errors_home,AB_home,RBI_home,BB_home,SO_home,PA_home,OBP_home,SLG_home,Starter_home,OBP_NUM_home,SLG_NUM_home,Runs_Mean_home,Hits_Mean_home,Errors_Mean_home,RBI_Mean_home,BB_Mean_home,SO_Mean_home,AB_Total_home,PA_Total_home,OBP_NUM_Total_home,SLG_NUM_Total_home,SLG_Mean_home,OBP_Mean_home,TeamAbr_home,Date_home,Run Line_home,Line Odds_home,Pitcher_home,ScoreDiff
0,5969899249830362306,Philadelphia Phillies,Atlanta Braves,2018-03-29 16:12:00,5,8,1,Away,6,1,31,5,6,11,38,0.342,0.355,Aaron Nola,12.996,11.005,NaN,NaN,NaN,NaN,NaN,NaN,31.0,38.0,12.996,11.005,NaN,NaN,PHI,329,-1.5,125.0,ANOLA-R,1,Home,9,0,34,7,4,6,40,0.359,0.559,Julio Teheran,14.360,19.006,NaN,NaN,NaN,NaN,NaN,NaN,34.0,40.0,14.360,19.006,NaN,NaN,ATL,329,1.5,-145.0,JTEHERAN-R,-3
1,6617391685886876920,Philadelphia Phillies,Atlanta Braves,2018-03-30 19:35:00,5,4,2,Away,10,0,40,5,5,10,48,0.319,0.450,Nick Pivetta,15.312,18.000,5.000000,6.000000,1.000000,5.000000,6.000000,11.000000,71.0,86.0,28.308,29.005,0.289605,0.419226,PHI,330,1.5,-225.0,NPIVETTA-R,2,Home,9,3,37,4,5,9,46,0.356,0.297,Mike Foltynewicz,16.376,10.989,8.000000,9.000000,0.000000,7.000000,4.000000,6.000000,71.0,86.0,30.736,29.995,0.475150,0.422353,ATL,330,-1.5,188.0,MFOLTYNEWICZ-R,1
2,701648310834011562,Philadelphia Phillies,Atlanta Braves,2018-03-31 19:10:00,2,15,3,Away,6,4,31,2,4,9,36,0.286,0.290,Vince Velasquez,10.296,8.990,5.000000,8.000000,0.500000,5.000000,5.500000,10.500000,102.0,122.0,38.604,37.995,0.337267,0.398704,PHI,331,1.5,-205.0,VVELASQUEZ-R,3,Home,19,0,43,14,5,5,51,0.490,0.674,Brandon McCarthy,24.990,28.982,6.000000,9.000000,1.500000,5.500000,4.500000,7.500000,114.0,137.0,55.726,58.977,0.348779,0.432901,ATL,331,-1.5,172.0,BMCCARTHY-R,-13
3,5221065161956846612,Washington Nationals,Atlanta Braves,2018-04-02 19:35:00,8,1,4,Away,10,1,37,8,10,10,48,0.438,0.432,Tanner Roark,21.024,15.984,7.000000,9.333333,0.333333,7.000000,4.666667,6.666667,141.0,171.0,65.010,70.990,0.447203,0.422942,WAS,402,-1.5,135.0,TROARK-R,4,Home,4,2,30,1,1,9,31,0.161,0.133,Sean Newcomb,4.991,3.990,9.000000,12.333333,1.000000,8.333333,4.666667,6.666667,144.0,168.0,60.717,62.967,0.430489,0.488825,ATL,402,1.5,-155.0,SNEWCOMB-L,7
4,6722752334276036486,Washington Nationals,Atlanta Braves,2018-04-03 19:47:00,6,13,5,Away,9,0,36,6,6,5,42,0.357,0.528,A.J. Cole,14.994,19.008,7.250000,9.500000,0.500000,7.250000,6.000000,7.500000,177.0,213.0,80.004,89.998,0.415146,0.461064,WAS,403,-1.5,120.0,ACOLE-R,5,Home,15,1,36,13,5,9,44,0.500,0.694,Julio Teheran,22.000,24.984,7.000000,10.250000,1.250000,6.500000,3.750000,7.250000,180.0,212.0,82.717,87.951,0.374804,0.421646,ATL,403,1.5,-140.0,JTEHERAN-R,-7
5,-6634706907505235000,Washington Nationals,Atlanta Braves,2018-04-04 12:11:00,1,7,6,Away,6,1,32,1,6,12,40,0.350,0.219,Max Scherzer,14.000,7.008,7.000000,9.400000,0.400000,7.000000,6.000000,7.000000,209.0,253.0,94.004,97.006,0.422526,0.452000,WAS,404,-1.5,-134.0,MSCHERZER-R,6,Home,7,0,32,6,4,11,36,0.306,0.438,Mike Foltynewicz,11.016,14.016,8.200000,11.200000,1.200000,7.800000,4.000000,7.600000,212.0,248.0,93.733,101.967,0.414863,0.459539,ATL,404,1.5,114.0,MFOLTYNEWICZ-R,-6
6,-8153093290743859869,Atlanta Braves,Colorado Rockies,2018-04-06 15:11:00,8,3,7,Away,11,0,36,8,7,2,44,0.409,0.528,Brandon McCarthy,17.996,19.008,8.000000,10.500000,1.000000,7.500000,4.000000,8.166667,248.0,292.0,111.729,120.975,0.411157,0.442137,ATL,406,1.5,-155.0,BMCCARTHY-R,8,Home,7,0,34,3,2,8,36,0.250,0.500,German Marquez,9.000,17.000,4.428571,8.000000,0.285714,4.428571,4.000000,9.142857,270.0,303.0,93.607

We now have to address some issues in which the odds data didn't match correctly with the game and team data.

In [154]:
pd.set_option('max_columns', None)
model_data[model_data['Run Line_home'] != -model_data['Run Line_away']]

,GameID,AwayTeam,HomeTeam,DateTime,AwayScore,HomeScore,GameNum_away,HomeAway_away,Hits_away,Errors_away,AB_away,RBI_away,BB_away,SO_away,PA_away,OBP_away,SLG_away,Starter_away,OBP_NUM_away,SLG_NUM_away,Runs_Mean_away,Hits_Mean_away,Errors_Mean_away,RBI_Mean_away,BB_Mean_away,SO_Mean_away,AB_Total_away,PA_Total_away,OBP_NUM_Total_away,SLG_NUM_Total_away,SLG_Mean_away,OBP_Mean_away,GameNum_home,HomeAway_home,Hits_home,Errors_home,AB_home,RBI_home,BB_home,SO_home,PA_home,OBP_home,SLG_home,Starter_home,OBP_NUM_home,SLG_NUM_home,Runs_Mean_home,Hits_Mean_home,Errors_Mean_home,RBI_Mean_home,BB_Mean_home,SO_Mean_home,AB_Total_home,PA_Total_home,OBP_NUM_Total_home,SLG_NUM_Total_home,SLG_Mean_home,OBP_Mean_home,ScoreDiff,TeamAbr_away,Date_away,Run Line_away,Line Odds_away,Pitcher_away,TeamAbr_home,Date_home,Run Line_home,Line Odds_home,Pitcher_home
0,2879045030533682763,Atlanta Braves,Philadelphia Phillies,2019-03-28 15:05:00-05:00,4,10,1,Away,7,2,30,4,6,9,36,0.361,0.367,Julio Teheran,12.996,11.010,NaN,NaN,NaN,NaN,NaN,NaN,30.0,36.0,12.996,11.010,NaN,NaN,1,Home,7,0,31,10,6,9,37,0.351,0.516,Aaron Nola,12.987,15.996,NaN,NaN,NaN,NaN,NaN,NaN,31.0,37.0,12.987,15.996,NaN,NaN,-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8187141795837815192,Atlanta Braves,Philadelphia Phillies,2019-03-30 16:05:00-05:00,6,8,2,Away,10,0,36,6,4,9,40,0.350,0.556,Bryse Wilson,14.000,20.016,4.000000,7.000000,2.000000,4.000000,6.000000,9.000000,66.0,76.0,26.996,31.026,0.305833,0.433200,2,Home,9,0,31,8,5,9,36,0.389,0.710,Nick Pivetta,14.004,22.010,10.000000,7.000000,0.000000,10.000000,6.000000,9.000000,62.0,73.0,26.991,38.006,0.432324,0.418935,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6891161377960279037,Atlanta Braves,Philadelphia Phillies,2019-03-31 19:05:00-05:00,1,5,3,Away,4,0,29,0,6,8,36,0.306,0.138,Kyle Wright,11.016,4.002,5.000000,8.500000,1.000000,5.000000,5.000000,9.000000,95.0,112.0,38.012,35.028,0.408237,0.409030,3,Home,5,0,27,5,9,8,39,0.410,0.444,Jake Arrieta,15.990,11.988,9.000000,8.000000,0.000000,9.000000,5.500000,9.000000,89.0,112.0,42.981,49.994,0.520630,0.435339,-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-5247093251011683673,Chicago Cubs,Atlanta Braves,2019-04-01 19:10:00-05:00,0,8,4,Away,9,6,33,0,4,5,37,0.351,0.364,Kyle Hendricks,12.987,12.012,9.333333,13.000000,0.333333,9.000000,6.000000,7.000000,147.0,172.0,71.422,77.001,0.481400,0.512588,4,Home,12,1,36,6,7,9,43,0.442,0.583,Sean Newcomb,19.006,20.988,3.666667,7.000000,0.666667,3.333333,5.333333,8.666667,131.0,155.0,57.018,56.016,0.312750,0.400126,-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-7338732862435343070,Chicago Cubs,Atlanta Braves,2019-04-03 19:20:00-05:00,4,6,5,Away,12,1,35,3,6,13,41,0.439,0.429,Jon Lester,17.999,15.015,7.000000,12.000000,1.750000,6.750000,5.500000,6.500000,182.0,213.0,89.421,92.016,0.447680,0.485864,5,Home,8,0,30,6,8,8,39,0.410,0.500,Julio Teheran,15.990,15.000,4.750000,8.250000,0.750000,4.000000,5.750000,8.750000,161.0,194.0,73.008,71.016,0.361394,0.435252,-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,2488215896629217713,Texas Rangers,Toronto Blue Jays,2018-04-28 16:08:00,7,4,28,Away,9,0,36,5,5,12,41,0.341,0.583,Bartolo Colon,13.981,20.988,3.703704,7.740741,0.851852,3.555556,3.148148,9.000000,935.0,1051.0,322.744,355.020,0.330725,0.343452,26,Home,7,1,34,4,1,4,35,0.229,0.559,Jaime Garcia,8.015,19.006,5.240000,7.720000,0.480000,4.920000,3.800000,8.960000,878.0,991.0,307.963,362.967,0.359792,0.355389,3,TEX,428.0,NaN,NaN,NaN,TOR,428.0,-1.5,163.0,MSTROMAN-R
2429,7599302588091231495,Texas Rangers,Toronto Blue Jays,2018-04-29 13:08:00,2,7,29,Away,6,1,32,2,0,12,32,0.188,0.344,Martin Perez,6.016,11.008,3.821429,7.785714,0.821429,3.607143,3.214286,9.107143,967.0,1083.0,328.760,366.028,0.337793,0.3451

In [17]:
odds.loc[(odds['Date']==527) & (odds['TeamAbr'] == 'CWS')]

,Date,Rot,VH,TeamAbr,Pitcher,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,Runs,Open,Close,Run Line,Line Odds,Open OU,Unnamed: 20,Close OU,Unnamed: 22
1581,527,914,H,CWS,INOVA-R,0,0,0,0,1,x,x,x,x,1,-115,-114,-1.5,160,9.5,-110,9.5,-115


In [ ]:
odds[(odds['TeamAbr']=='ATL')].iloc[4]

In [ ]:
team_data[(team_data['TeamAbr']=='LAD')].count()

In [ ]:
game_level[game_level['GameID']==1096196326745839632]

In [ ]:
game_level[game_level['HomeTeam']=='Atlanta Braves'].iloc[0]['DateTime']

In [ ]:
box_score = scores[3]

In [ ]:
pd.to_datetime(dateparser.parse(box_score.date + ' ' + box_score.time.replace('Local', '')))

In [ ]:
dateparser.parse(box_score.date + ' ' + box_score.time.replace('Local', ''))